In [1]:
from google.colab import drive
drive.mount("/content/drive")

!pip install datasets evaluate transformers[sentencepiece]

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 36.3 MB/s 
     |████████████████████████████████| 72 kB 1.8 MB/s 
     |████████████████████████████████| 5.5 MB 62.7 MB/s 
     |████████████████████████████████| 212 kB 100.1 MB/s 
     |████████████████████████████████| 115 kB 92.9 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 127 kB 78.0 MB/s 
     |████████████████████████████████| 7.6 MB 64.7 MB/s 
     |████████████████████████████████| 1.3 MB 80.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import logging
import math
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional 

import datasets
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    is_torch_tpu_available,
    set_seed,
)

from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

In [7]:
!python run_clm.py \
    --model_name_or_path gpt2 \
    --dataset_name wikitext \
    --dataset_config_name wikitext-2-raw-v1 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --do_train \
    --do_eval \
    --output_dir /content/test-clm

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_me

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

PATH = "/content/test-clm"
model = AutoModelForCausalLM.from_pretrained(PATH)
tokenizer = AutoTokenizer.from_pretrained(PATH)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer) 

In [9]:
generator("I want to be")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'I want to be able to show what I mean here by not having to have my hair tied down. [ She ] really looks like that girl around here. And I feel like I could have turned that off and left it on for a while,'}]